In [2]:
import cv2
import numpy as np 
import dlib
import check_in_out
from get_data import get_data
import tkinter as tk
import pickle 
from PIL import Image, ImageTk
import time
from datetime import datetime, timedelta
from scipy import stats
from check_in_out import check_in_out

class App:
    def __init__(self, window, window_title, video_source=0):
        self.window = window
        self.window.title(window_title)
        self.video_source = video_source
        self.name = ""
        self.status = ""
        self.time_check = ""   
        self.image = ""
        self.status = ""
        self.time_check = ""
        self.name_data = get_data().getDataName()

        self.cap = MyVideoCapture(self.video_source)
        self.time_label = tk.Label(window, font=('Helvetica', 18))
        self.time_label.grid(row=0, column=0, padx=10, pady=10)
        self.canvas = tk.Canvas(window, width=self.cap.width, height=self.cap.height)
        self.canvas.grid(row=1, column=0, padx=10, pady=10)

        self.delay = 15
        self.decision_name = []
        self.name_list = get_data().getNameList()
        self.facedetection = cv2.CascadeClassifier('path\\haarcascade_frontalface_default.xml')
        self.detector = dlib.get_frontal_face_detector()
        self.sp = dlib.shape_predictor('path\\shape_predictor_68_face_landmarks.dat')
        self.model = dlib.face_recognition_model_v1('path\\dlib_face_recognition_resnet_model_v1.dat')
        self.FACE_DESC, self.FACE_NAME = pickle.load(open('path\\trainset.pk', 'rb'))

        self.color_change_end_time = None

        self.images = get_data().getImage()
        self.photo = None  # Initialize the photo attribute
        self.update()
        self.update_time()
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.window.mainloop()
    
    def update(self):
        ret, frame = self.cap.get_frame()
        if ret:
            frame = frame[:, 200:700]
            frame = cv2.flip(frame, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.facedetection.detectMultiScale(gray, 1.3, 5)

            if self.color_change_end_time and datetime.now() < self.color_change_end_time:
                frame[:380, :] = [80, 255, 80]
                frame[40:340, 50:350] = self.image
                frame[380:, :] = [255, 255, 255]
                cv2.putText(frame, self.name, (50, 410), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                cv2.putText(frame, self.status, (80, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
                cv2.putText(frame, self.time_check, (220, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
            else:
                self.color_change_end_time = None
                for (x, y, w, h) in faces:
                    face = frame[y-10:y+h+10, x-10:x+w+10][:,:,::-1]
                    dets = self.detector(face, 1)
                    for k, d in enumerate(dets):
                        shape = self.sp(face, d)
                        face_desc0 = self.model.compute_face_descriptor(face, shape, 1)
                        dist = []
                        for face_desc in self.FACE_DESC:
                            dist.append(np.linalg.norm(np.array(face_desc) - np.array(face_desc0)))
                        dist = np.array(dist)
                        idx = np.argmin(dist)
                        if dist[idx] < 0.5:
                            name = self.FACE_NAME[idx]
                            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                            self.decision_name.append(name)

                if len(self.decision_name) == 3:
                    popular_name = stats.mode(self.decision_name)[0][0]
                    if popular_name == "Unknow":
                        self.decision_name.clear()
                        self.name = "          Unknow"
                        self.color_change_end_time = datetime.now() + timedelta(seconds=5)
                        self.image = self.images[-1]
                    else:
                        id = np.where(self.name_list == popular_name)[0][0]
                        print(id)
                        data = check_in_out().save_data(id)
                        self.status, self.time_check = data[0], data[1]
                        print(data)
                        self.decision_name.clear()
                        self.name = str(self.name_data[id][0]) + " " + self.name_data[id][1]
                        self.image = self.images[id]
                        self.color_change_end_time = datetime.now() + timedelta(seconds=5)
                    
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(frame))
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        self.window.after(self.delay, self.update)
    
    def update_time(self):
        current_time = time.strftime("%H:%M:%S")
        self.time_label.config(text=current_time)
        self.window.after(1000, self.update_time)
    
    def on_closing(self):
        self.cap.release()
        self.window.destroy()

class MyVideoCapture:
    def __init__(self, video_source=0):
        self.cap = cv2.VideoCapture(video_source)
        if not self.cap.isOpened():
            raise ValueError("Unable to open video source", video_source)
        self.width = 400
        self.height = self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def get_frame(self):
        if self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                return (ret, None)
        else:
            return (False, None)

    def release(self):
        if self.cap.isOpened():
            self.cap.release()

app = App(tk.Tk(), "Face Recognition")

0
['Checked in' '10:20:18']
0
['Checked out' '10:20:24']
